TODO:
- zrobić jakiś indentyfikator na entery i zamieniać \<br\> na ten identyfikator, przy ostatecznym wypisywaniu zmieniać identyfikator na entery

In [169]:
%autocall 1
import requests
import re
from nltk import ngrams
from nltk.tokenize.moses import MosesTokenizer
import datetime 

ADRES = "https://www.tekstowo.pl"
EMPTY_STRING_LENGTH = 6
ENTER = "enterenter"

Automatic calling is: Smart


In [112]:
def extract_song_url_from_line(line):
    return re.findall("piosenka.+html", line)[0]

def find_all_subpages(html_source):
    return [re.findall("/piosenki_artysty.+html",line)[0] 
            for line in html_source 
            if 'alfabetycznie' in line and 'strona' in line]

def get_songs_urls(artists):
    artist_songs = {}
    for artist in artists:
        artist_url = "https://www.tekstowo.pl/piosenki_artysty,"+artist+".html"
        r = requests.get(artist_url).text.split('\n')
        
        subpages = set(find_all_subpages(r))
        
        songs_urls=set()
        for subpage in subpages:
            r = requests.get(ADRES+subpage)
            not_yet = True
            for line in r.text.split('\n'):
                if not_yet and 'przeboje' not in line:
                    continue
                not_yet=False
                if 'html' in line and artist in line.lower() and 'piosenka' in line:
                        songs_urls.add(ADRES + '/' + extract_song_url_from_line(line.strip()))
        
        artist_songs[artist]=songs_urls
    return artist_songs               

In [ ]:
get_songs_urls(['enej', 'lzy'])

In [170]:
def extract_lyrics(urls):
    polisher = lambda x: x.encode('iso-8859-1').decode('utf-8')
    
    songs = []
    for url in urls:
#         print 'parsing', url
        html_source = requests.get(url).text
        not_yet = True
        no_more = False
        lyrics = []
        for line in html_source.split('\n'):
            if 'song-text' in line:
                not_yet = False
            if not not_yet and 'javascript' in line:
                no_more = True
            if not not_yet and not no_more and len(line)>EMPTY_STRING_LENGTH:
                try :
                    lyrics.append(polisher(line.strip()).replace('<br />', ENTER).lower())
                except UnicodeDecodeError:
                    print line, line.__repr__()
        songs.append('\n'.join(lyrics[2:-2]))
    return songs

In [171]:
html = 'https://www.tekstowo.pl/piosenka,czerwone_gitary,przesta__wodzic_mnie_za_nos.html'
extract_lyrics([html,])

[u'mam tego do\u015b\u0107! enterenter\nmam tego do\u015b\u0107! enterenter\ni zn\xf3w si\u0119 rozgl\u0105dasz, enterenter\nu\u015bmiechasz do kogo\u015b - enterenter\njeste\u015b niemo\u017cliwa wprost! enterenter\ntak jakbym nie istnia\u0142 enterenter\ni nie sta\u0142 tu\u017c obok. enterenter\nprzesta\u0144 wodzi\u0107 mnie za nos! enterenter\nenterenter\nmam tego do\u015b\u0107! enterenter\nmam tego do\u015b\u0107! enterenter\nzn\xf3w siedzisz w jakiego\u015b enterenter\ngwiazdora wpatrzona, enterenter\nzas\u0142uchana w jego g\u0142os. enterenter\ni marzysz na pewno enterenter\no jego ramionach; enterenter\nfilmu mam ju\u017c tak\u017ce do\u015b\u0107! enterenter\nenterenter\nenterenter\nmam tego do\u015b\u0107! enterenter\nmam tego do\u015b\u0107! enterenter\ndzi\u015b rzuc\u0119 to wszystko enterenter\ni p\xf3jd\u0119 w \u015bwiat sobie, enterenter\npotem mnie o powr\xf3t pro\u015b. enterenter\nnie b\u0119d\u0119 si\u0119 waha\u0142 enterenter\ni zaraz to zrobi\u0119; enterent

In [172]:
def download_lyrics(artists):
    song_book = {}
    for artist in artists:
        print datetime.datetime.now(), artist
        song_book[artist] = extract_lyrics(get_songs_urls([artist,])[artist])
    return song_book

dataset_orig = download_lyrics(['enej', 'lzy', 'happysad', 'kayah', 'czerwone_gitary',
                                'pidzama_porno', 'krzysztof_krawczyk', 'stare_dobre_malze_stwo',
                                'zabili_mi_zolwia', 'elektryczne_gitary', 'monika_brodka',
                                'hey', 'gaba_kulka', 'coma', 'ryszard_rynkowski', 'natalia_kukulska'])

2017-11-24 23:31:48.473570 enej
2017-11-24 23:33:05.612320 lzy
2017-11-24 23:35:00.887549 happysad
2017-11-24 23:36:38.901054 kayah
u'\xc5\xbbe wci\xc4\x85\xc5\xbc szumisz tylko pustk\xc4\x85\r'
u'bo nie ludzie s\xc5\x82owa, ale s\xc5\x82owa ludzi nios\xc4\x85\r'
2017-11-24 23:38:33.501543 czerwone_gitary
2017-11-24 23:41:48.304188 pidzama_porno
u'kt\xc3\xb3re szybko odje\xc5\xbcd\xc5\xbcaj\xc4\x85\r'
u'Jedn\xc4\x85 spocon\xc4\x85 noc\xc4\x85\r'
2017-11-24 23:43:31.622233 krzysztof_krawczyk
u'Co nam do dzisiaj brzmi\xc4\x85\r'
u'Cuda, cuda og\xc5\x82aszaj\xc4\x85\r'
u'nie by\xc5\x82o, nigdy nie, nie by\xc5\x82o tak.\xc2\xa0\r'
2017-11-24 23:46:29.515990 stare_dobre_malze_stwo
u'nad Krak\xc3\xb3w i wie\xc5\xbc\xc4\x99 Mariack\xc4\x85\r'
u'\xc5\xbbe ich \xc5\x9blady prosto do nieba prowadz\xc4\x85 \r'
u'zn\xc3\xb3w zej\xc5\x9b\xc4\x87 na nasz\xc4\x85 ziemi\xc4\x99 \xc5\x9bwi\xc4\x99t\xc4\x85\r'
u'Anio\xc5\x82y na poddaszu p\xc5\x82acz\xc4\x85\r'
u'grzesznic\xc4\x85 czy \xc5\x9bwi\xc4\x99

In [173]:
def preprocess(data_dict, ngram=3):
    # zwraca listę z piosenkami, każda piosenka to lista ngramów, ngramy to tuple
    dataset = []
    tokeniser = MosesTokenizer()
    for band in data_dict:
        for i in range(len(data_dict[band])):
            song = tokeniser.tokenize(data_dict[band][i], return_str=False)
            n_song = list(ngrams(song, ngram))
            dataset.append(n_song)
    return dataset

# small = {}
# small['e'] = dataset_orig['enej'][2:4]
dataset = preprocess(dataset_orig)

In [174]:
def populate_markov(dataset):
    mm={}
    for song in dataset:
        for i in range(len(song)-1):
            current = song[i]
            following = song[i+1]
            if current not in mm:
                mm[current]={}
            if following not in mm[current]:
                mm[current][following] = 0
            mm[current][following] += 1
    return mm

mm = populate_markov(dataset)

In [175]:
alle = 0
fancy = 0
for key in mm:
    alle += 1
    if len(mm[key])>=2:
        # print mm[key]
        fancy += 1
            
print float(fancy)/alle

0.0639950780081


In [176]:
import numpy as np
def generate(model, text = "to jest", length=100):
    current = tuple(text.split())
    for i in range(length):
        highest_probability = 0
        possibilities = []
        for follower in model[current]:
            if model[current][follower] == highest_probability:
                possibilities.append(follower)
            if model[current][follower] > highest_probability:
                possibilities = [follower, ]
        the_chosen_one = possibilities[np.random.randint(len(possibilities))]
        print the_chosen_one[-1]
        current = the_chosen_one

In [246]:
def unlikely_generate(model, text = "to jest", length=100):
    current = tuple(text.split())
    tek=' '.join(current)
    try:
        for i in range(length):
            highest_probability = 0
            possibilities = []
            for follower in model[current]:
                possibilities.append(follower)
            the_chosen_one = possibilities[np.random.randint(len(possibilities))]
            tek+= ' ' + the_chosen_one[-1]
            current = the_chosen_one
    except KeyError:
        return tek
    return tek

In [249]:
generated =  unlikely_generate(mm, u"ja i ty")
print generated.replace(ENTER, '\n')

ja i ty ... 
 
 bądź lekarstwem mym 
 
 można się kochać 
 będziesz tulił mnie i całował jak dawniej 
 i będę cię prosić , bym mogła zostać do rana 
 wystarczy nam 
 
 nie musi być też taki przystojny , 
 ale ważne jest to , co najlepszego masz
 raz ty , raz ja.
 jeszcze tyle miejsc , gdzie pragną tańczyć.
 
 nie zmarnujmy tej ostatniej szansy -
 koncert musi trwać , nuty trzeba znać
 i grać , i grać.
 
 jeszcze mamy czas uwierzyć w siebie.
 jeszcze bóg nam da niechciane szczęście.
 jeszcze drogi dwie , w


In [252]:
import json
with open('dataset.json', 'w') as f:
    json.dump(dataset_orig, f)